# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-05 01:31:19] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-05 01:31:19] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-05 01:31:19] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:31:19] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:31:22] WARNING server_args.py:1165: Attention backend not explicitly specified. Use fa3 backend by default.


...

Capturing batches (bs=120 avail_mem=57.09 GB):   5%|▌         | 1/20 [00:00<00:03,  5.83it/s]

Capturing batches (bs=72 avail_mem=57.06 GB):  35%|███▌      | 7/20 [00:00<00:00, 20.36it/s]

Capturing batches (bs=32 avail_mem=57.04 GB):  50%|█████     | 10/20 [00:00<00:00, 22.30it/s]

Capturing batches (bs=8 avail_mem=57.02 GB):  80%|████████  | 16/20 [00:00<00:00, 19.42it/s] 

Capturing batches (bs=1 avail_mem=57.01 GB): 100%|██████████| 20/20 [00:00<00:00, 20.09it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Alice. I'm a nice girl. I like ________ very much. [ ]
A. apples
B. an apple
C. apple
D. a apple
Answer: C

What is the last letter of the name Alice? [ ]
A. A
B. E
C. I
D. O
Answer: A

What is the last letter of the name Alice? [ ]
A. A
B. E
C. I
D. O
Answer: A

How many letters are there in the word "baby"? [ ]
A. 4
B. 5
C.
Prompt: The president of the United States is
Generated text:  a minister of the government, while a minister of the government is a minister of the executive branch, so the president is a minister of the executive branch. I want to know what can be the noun category here? To determine the noun category in the given statement, we need to carefully analyze the terms involved and their definitions. Let's break down the statement:

1. **President of the United States**: A person in the position of President of the United States.
2. **Minister of the government**: A person who holds a position in the government.
3. **Minister

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a short description of your character, such as "funny, adventurous, or curious"] and I enjoy [insert a short description of your interests or hobbies, such as "reading, traveling, or cooking"]. I'm always looking for new experiences and challenges, and I'm always eager to learn and grow. What's your favorite hobby or activity? I'm always looking for new ways to challenge myself and expand my knowledge. What's

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in France and the second-largest city in the European Union. Paris is known for its rich history, beautiful architecture, and vibrant culture. It is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also a major economic and cultural center, hosting many world-renowned events and festivals. It is a popular tourist destination and a major hub for international business and trade. Paris is a city that has been a center of culture and politics for centuries, and continues to be a major influence on French society and politics today.

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. Here are some possible future trends in AI:

1. Increased focus on ethical considerations: As AI becomes more integrated into our daily lives, there will be a growing emphasis on ethical considerations. This will include issues such as bias, transparency, and accountability.

2. More integration with human decision-making: AI will become more integrated into decision-making processes, allowing humans to make better decisions based on data and information.

3. Greater use of AI in healthcare: AI will be used to improve the accuracy and speed of medical diagnoses, as well



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name]. I'm a [role] who loves [what you do best]. I'm [age] years old and I currently [occupation]. I enjoy [reason why you like what you do]. I'm passionate about [what you love to do]. My favorite hobby is [anything], but [justify the reason why]. I'm patient and kind, always ready to help others. I love [what you do best] because [why]. I hope to achieve [short-term or long-term goal]. I’m happy to share my experiences and skills with anyone who chooses to learn from me. I'm [any other relevant

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

In Paris, you can find the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. 

The city is known for its historical significance and浪漫氛围，with a rich cultural heritage and a lively atmosphere. 

Paris is a popular tourist destination, known for its a

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

]

...

,

 and

 that

 ethical

 guidelines

In [6]:
llm.shutdown()